# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [1]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
dsms = DSMS(env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [3]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.Testingmachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.Testingmachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.5,
        "Length": [0.1, 0.2],
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.8,
        "Length": [0.8, 0.9],
    }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1,item2],
    annotations=["www.researchBACiri.org/foo"],
)

dsms.commit()

/app/dsms/knowledge/kitem.py:570: UserWarning: A flat dictionary was provided for custom properties.
                    Will be transformed into `KItemCustomPropertiesModel`.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Producer`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Room Number`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Description`. Cannot check if value is of correct type.
  warnings.warn(



<b> <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>    </b> 


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [4]:
result = dsms.search(ktypes=[dsms.ktypes.Specimen], limit=2)
print(result)
print("Name of the first kitem:")
print(result.hits[0].kitem.name)

hits:
- kitem:
    name: Specimen123
    id: 54ff62d5-38f7-474c-856e-fa9c8be96a7c
    ktype_id: specimen
    slug: specimen123-54ff62d5
    annotations: []
    attachments:
    - name: subgraph.ttl
      content: null
    linked_kitems: []
    affiliations: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    avatar_exists: false
    contacts: []
    created_at: 2025-01-17 10:43:36.912252
    updated_at: 2025-01-17 10:43:36.912252
    external_links: []
    kitem_apps: []
    user_groups: []
    custom_properties:
      sections:
      - id: ida3c5c42685526
        name: Untitled Section
        entries:
        - id: id175f885f8dda58
          type: Slider
          label: Width
          value: 0.5
          measurement_unit:
            iri: http://qudt.org/vocab/unit/MilliM
            label: Millimetre
            namespace: http://qudt.org/vocab/unit
        - id: idef5a37328789f
          type: Slider
          label: Length
          value:
          - 0.1
  

... and for all of type `Organization` and `Testingmachine`:

In [6]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.Testingmachine], limit=1):
    print(result.kitem)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

kitem:
  name: Research Institute ABC
  id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
  ktype_id: organization
  slug: researchinstituteabc-56a1243e
  annotations:
  - iri: www.researchBACiri.org/foo
    label: foo
    namespace: www.researchBACiri.org
  attachments: []
  linked_kitems:
  - id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
    name: Machine-1
    slug: machine-1-7d04a850
    ktype_id: testingmachine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
    - id: 75b7126b-950d-4473-b86c-f9780dae333f
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []
    custom_properties:
      sections:
      - id: id1737

... or for all of type `Specimen` with `Specimen1` in the name:

In [10]:
for result in dsms.search(query="Specimen1", ktypes=[dsms.ktypes.Specimen], limit=1):
    print(result)

kitem:
  name: Specimen123
  id: 54ff62d5-38f7-474c-856e-fa9c8be96a7c
  ktype_id: specimen
  slug: specimen123-54ff62d5
  annotations: []
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems: []
  affiliations: []
  authors:
  - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
  avatar_exists: false
  contacts: []
  created_at: 2025-01-17 10:43:36.912252
  updated_at: 2025-01-17 10:43:36.912252
  external_links: []
  kitem_apps: []
  user_groups: []
  custom_properties:
    sections:
    - id: ida3c5c42685526
      name: Untitled Section
      entries:
      - id: id175f885f8dda58
        type: Slider
        label: Width
        value: 0.5
        measurement_unit:
          iri: http://qudt.org/vocab/unit/MilliM
          label: Millimetre
          namespace: http://qudt.org/vocab/unit
      - id: idef5a37328789f
        type: Slider
        label: Length
        value:
        - 0.1
        - 0.2
        measurement_unit:
          iri: http://qudt.org/vocab/unit/Mi

... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [16]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], limit=1
    ):
    print(result)


## 5.3. Fetching linked KItems from a KItem

In the beginning under **5.1** we created some kitems and linked each other. Now we want to fetch the linked kitems and display them to the user. For this we use the `linked_kitems` attribute.

In [17]:
item5.linked_kitems

[linked_kitem:
   id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
   name: Machine-1
   slug: machine-1-7d04a850
   ktype_id: testingmachine
   summary: null
   avatar_exists: false
   annotations:
   - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
     label: TestingMachine
     namespace: https://w3id.org/steel/ProcessOntology
   linked_kitems:
   - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
   - id: 75b7126b-950d-4473-b86c-f9780dae333f
   external_links: []
   contacts: []
   authors:
   - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
   linked_affiliations: []
   attachments:
   - name: subgraph.ttl
     content: null
   user_groups: []
   custom_properties:
     sections:
     - id: id17371114692064uv9g3
       name: Misc
       entries:
       - id: id1737111469206ueowwr
         type: Text
         label: Producer
         value: TestingLab GmBH
         measurementUnit: null
         relationMapping: null
       - id: id1737111469206dllz5f
         type: Text
         label

However, this linked KItem is not a "real" KItem. Due to performance reasons, we only display a slim representation of the linked KItem. Imagine if the linked KItem also has a linked KItem... this might lead to an infinite recursion. We can fetch the "real" KItem by using the `fetch` method of the linked KItem:


In [18]:
item5.linked_kitems[0].fetch()

/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Producer`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Room Number`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Description`. Cannot check if value is of correct type.
  warnings.warn(


kitem:
  name: Machine-1
  id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
  ktype_id: testingmachine
  slug: machine-1-7d04a850
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
    name: Research Institute ABC
    slug: researchinstituteabc-56a1243e
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
    - id: 9c663adb-6173-4cb5-acc1-9d89d37b7155
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-17T10:57

We can also get the linked KItems grouped by annotation:

In [19]:
item5.linked_kitems.by_annotation

{'https://w3id.org/steel/ProcessOntology/TestingMachine': [linked_kitem:
    id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
    name: Machine-1
    slug: machine-1-7d04a850
    ktype_id: testingmachine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
    - id: 75b7126b-950d-4473-b86c-f9780dae333f
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []
    custom_properties:
      sections:
      - id: id17371114692064uv9g3
        name: Misc
        entries:
        - id: id1737111469206ueowwr
          type: Text
          label: Producer
          value: TestingLab GmBH
          measurementUnit: null
          

Again, we are able to fetch the "real" KItem by using the `fetch` method of the linked KItem:

In [20]:
item5.linked_kitems.by_annotation["https://w3id.org/steel/ProcessOntology/TestingMachine"][0].fetch()

kitem:
  name: Machine-1
  id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
  ktype_id: testingmachine
  slug: machine-1-7d04a850
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
    name: Research Institute ABC
    slug: researchinstituteabc-56a1243e
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
    - id: 9c663adb-6173-4cb5-acc1-9d89d37b7155
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-17T10:57

Additionally, we can retrieve the linked KItems grouped by ktype:

In [21]:
item5.linked_kitems.by_ktype

{ktype:
   id: testingmachine
   name: Testing machine
   context: false
   context_schema: []
   webform:
     semantics_enabled: true
     sections_enabled: false
     class_mapping:
     - https://w3id.org/pmd/co/Specimen
     sections:
     - id: id18d888a9c9fa48
       name: Untitled Section
       inputs:
       - id: ida450e96a27a6e8
         label: Machine
         widget: Knowledge item
         required: false
         value: null
         hint: null
         hidden: false
         ignore: false
         select_options: []
         measurement_unit: null
         relation_mapping:
           iri: https://w3id.org/steel/ProcessOntology/hasMachine_Object
           type: object_property
           class_iri: https://w3id.org/steel/ProcessOntology/Machine
         relation_mapping_extra: null
         multiple_selection: false
         knowledge_type: testingmachine
         range_options: null
         placeholder: null
       - id: idba8e853b2fdbe
         label: Specimen
    

... and retrieve them by type:

In [23]:
item5.linked_kitems.by_ktype[dsms.ktypes.Testingmachine][0].fetch()

kitem:
  name: Machine-1
  id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
  ktype_id: testingmachine
  slug: machine-1-7d04a850
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: 56a1243e-0e8e-4528-bad0-b683b84b64e7
    name: Research Institute ABC
    slug: researchinstituteabc-56a1243e
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 7d04a850-f4e5-4682-b043-1414cf4d0fd9
    - id: 9c663adb-6173-4cb5-acc1-9d89d37b7155
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-17T10:57

Clean up the DSMS from the tutortial:

In [14]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()